#TweetDownload

Downloads tweets related to medical symptoms

In [1]:
import pickle
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import time
import gzip
import sys

#Credentials

In [2]:
consumer_key=""
consumer_secret=""

access_token=""
access_token_secret=""

#Code

In [3]:
def get_name():
    global gdatadir
    global gprefix
    t = int(time.time()*1000000)
    if gdatadir:
        return "{}{}{}.json.gz".format(gdatadir,gprefix,t)
    else:
        return "{}{}.json.gz".format(gprefix,t)

In [4]:
class MyListener(StreamListener):
    """ A listener handles tweets are the received from the stream.
    This listener writes out tweets in json format in groups of 1000 prt file
    """

    def __init__(self):
        self.fd = False
        self.count = 0
        self.max_tweets = 10000

    def on_data(self, data):
        if self.fd and self.count > self.max_tweets:
            self.fd.close()
            self.fd = False
        if not self.fd:
            print '*',
            self.fd = gzip.open(get_name(), 'wb')
            self.count = 0
        #print(data)
        self.fd.write(data)
        self.count += 1
        return True

    def on_error(self, status):
        print("error:{}".format(status))

    def on_timeout(self):
        print("connection dropped")
        if self.fd:
            self.fd.close()
            self.fd = False
        raise Exception("dropped connection")

In [5]:
def tweet_dump(track=False, count=10, delay=60, datadir=False, prefix=''):
    print 'starting tweet dump'
    sys.stdout.flush()
    global gdatadir
    global gprefix
    gdatadir = datadir
    gprefix = prefix
    l = MyListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    while not count or count > 0:
        try:
            stream = Stream(auth, l)
            if track:
                stream.filter(track=track)
            else:
                stream.sample()
        except Exception,e:
            print 'Exception: ', e
            if e != "dropped connection":
                return
        time.sleep(delay)
        if count:
            count -= 1

#Analysis

In [ ]:
combined_symptoms = pickle.load(open( "symptoms.p", "rb" ))
print len(combined_symptoms)

162


In [ ]:
tweet_dump(combined_symptoms)
#tweet_dump()

starting tweet dump
